## 단어 벡터
- 단어의 의미 또는 유사도 계산
- Word Embeddings라고 부르기도 함

## 단어의 의미를 벡터로 만들기

### Word2Vec
- Google 연구자 토머스 미코로프가 만든 방법
- 딥러닝 기술을 사용해 단어를 벡터로 만드는 방법
    - 단어는 주변의 단어들과 관계가 있다는 점에 착안하여 의미를 추축
    - 약점은 반대되는 의미를 가진 단어를 비슷한 벡터로 표현 (좋다, 싫다 등)
- Skip-gram, CBOW가 있음

### 코퍼스
- 목적에 맞게 적절한 문장을 학습시켜야 함
- '존나'의 경우, 과거에는 부정적인 느낌이었으나 최근에는 아님
- 코퍼스 : 모델을 만들기 위한 대량의 띄어쓰기로 구분된 데이터를 포함하여, 컴퓨터로 검색이 가능한 대량의 언어 데이터

### 코퍼스 생성하기
- [위키피디아(한국어) 데이터](https://dumps.wikimedia.org/kowiki/latest/) - [kowiki-latest-pages-articles.xml.bz2](https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2)
- Docker 등 linux 명령 프롬프트 창을 이용하여 압축 풀기  
    `bzip2 -d kowiki-latest-pages-articles.xml.bz2`
- Ruby 설치
- wp2txt 설치  
    `gem install wp2txt`
- XML to txt  
    `wp2txt --input-file ./kowiki-latest-pages-articles.xml`
- 분리된 텍스트 파일을 하나의 파일로 뭉치기  
    `cat kowiki-latest-pages-articles-* > wiki.wp2txt`

In [1]:
%%time

import codecs
from konlpy.tag import Okt

# 파일 열기
readFp = codecs.open('../datasets/wiki.wp2txt', 'r', encoding='utf8')
gubun_file = '../datasets/wiki.gubun'
writeFp = open(gubun_file, 'w', encoding='utf8')

# 형태소 분석
okt = Okt()
i = 0
# 텍스트 한 줄씩 처리
while True:
    line = readFp.readline()
    if not line: break
    if i%1000000 == 0:
        print('current -', i)
    i += 1
    
    # 형태소 분석
    malist = okt.pos(line, norm=True, stem=True)
    
    # 필요한 어구만 대상으로 하기
    for word in malist:
        # 어미/조사/구두점 제외
        if not word[1] in ['Josa', 'Eomi', 'Punctuation']:
            writeFp.write(word[0] + ' ')
writeFp.close()

current - 0
current - 1000000
current - 2000000
current - 3000000
current - 4000000
current - 5000000
current - 6000000
current - 7000000
current - 8000000
current - 9000000
current - 10000000
current - 11000000
current - 12000000
current - 13000000
current - 14000000
current - 15000000
current - 16000000
current - 17000000
current - 18000000
current - 19000000
current - 20000000
current - 21000000
current - 22000000
current - 23000000
Wall time: 4h 33min 24s


결과 : 23,000,000+, 4시간 30분, 1.06GB

### 자연어 처리 라이브러리 Gensim
- 주제 분석 등 자연어 처리 가능

### 모델 만들기
- `gensim.models.word2vec.Word2Vec(sentences, sg, vector_size, window)` : 모델 생성
    - `sg` : Word2Vec에서 사용할 알고리즘으로 1=Skip-gram, 0=CBOW
    - `vector_size` : 벡터의 차원 설정. 100~200 정도를 설정하는 것이 일반적이지만, 숫자가 늘어날수록 오랜 시간이 걸림
    - `window` : 학습할 단어를 연관시킬 앞뒤의 단어 수 지정.

In [3]:
%%time

from gensim.models import word2vec

# 코퍼스 읽어 들이기
sentences = word2vec.Text8Corpus('../datasets/wiki.gubun')

# 모델 만들기
model = word2vec.Word2Vec(sentences, sg=1, vector_size=200, window=5)

# 모델 저장
model.save('../models/wiki.model')

Wall time: 39min 12s


### 모델을 사용해 계산하기

#### 유의어

In [4]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('../models/wiki.model')
results = model.wv.most_similar(positive=['과자'])
for result in results:
    print(result)

('과자류', 0.7754049897193909)
('화과자', 0.7620612978935242)
('주먹밥', 0.7221462726593018)
('덴푸라', 0.7182508111000061)
('계란말이', 0.7110555171966553)
('젤라또', 0.7085745334625244)
('케이크', 0.7069568037986755)
('사탕', 0.7049573659896851)
('군것질', 0.7046226859092712)
('디저트', 0.7035354375839233)


#### 반의어?

In [5]:
results = model.wv.most_similar(positive=['왕자', '여성'], negative=['남성'])
for result in results:
    print(result)

('왕세자', 0.6083212494850159)
('왕비', 0.5947650074958801)
('1394~1460', 0.5590384006500244)
('아브틴', 0.5570813417434692)
('왕녀', 0.5559399724006653)
('코넛', 0.550190269947052)
('공주', 0.5485385060310364)
('여왕', 0.5427746176719666)
('1398~1479', 0.5278303623199463)
('스트래선', 0.523162305355072)


#### 문장 유사도

In [6]:
from konlpy.tag import Okt
from gensim.models import word2vec

# 모델 읽어 들이고 형태소 분석 준비
model = word2vec.Word2Vec.load('../models/wiki.model')
okt = Okt()

def print_emargency(text):
    print(text)
    # 전달된 문장을 형태소 분석하기
    node = okt.pos(text, norm=True, stem=True)
    for word, form in node:
        # 필요한 형태소만 추출
        if form == 'Noun' or form == 'Verb' or form == 'Adjective' or form == 'Adverb':
            # 급하다와 비슷한 단어
            print('-', word, ':', model.wv.similarity(word, '급하다'))
            
print_emargency('컴퓨터에 문제가 생겼어요. 빨리 해결해야 하는 문제가 있어서 지원 요청합니다.')
print_emargency('사용 방법을 잘 모르겠습니다.')

컴퓨터에 문제가 생겼어요. 빨리 해결해야 하는 문제가 있어서 지원 요청합니다.
- 컴퓨터 : 0.0993922
- 문제 : 0.35688016
- 생기다 : 0.38950863
- 빨리 : 0.5986096
- 해결 : 0.3844186
- 하다 : 0.38777393
- 하다 : 0.38777393
- 문제 : 0.35688016
- 있다 : 0.34192967
- 지원 : 0.27302924
- 요청 : 0.43129143
- 하다 : 0.38777393
사용 방법을 잘 모르겠습니다.
- 사용 : 0.23760146
- 방법 : 0.2737294
- 자다 : 0.29164606
- 모르다 : 0.32622546
